In [1]:
from hello import hello, hello3
from collections import OrderedDict

In [16]:
import os
path = "E:\\KLS Files"
def fullpath(f):
    return os.path.join(path, f)
def join(*args):
    return os.path.join(path, *args)

In [47]:
dfp = "pbs %s def"
def alms(fp):
    for f in os.listdir(path):
        if f.startswith(fp):
            fp = f
            break
    else:
        raise FileNotFoundError(f)

    if fp.startswith("0003"):
        sz = "3"
    elif fp.startswith("0015"):
        sz = "15"
    elif fp.startswith("0080"):
        sz = "80"
    else:
        raise ValueError(fp)
    refpth = join(dfp%sz, "LabVIEW Data", "Alarms On.alm")
    oldpth = join(fp, "LabVIEW Data", "Alarms On.cfg")
    print("Alarms On:")
    compare_alms(refpth, oldpth)
    print()
    print("Alarms Off:")
    refpth = join(dfp%sz, "LabVIEW Data", "Alarms Off.alm")
    oldpth = join(fp, "LabVIEW Data", "Alarms Off.cfg")
    compare_alms(refpth, oldpth)
    
def open_alm(fp):
    with open(fp, 'r') as f:
        f.readline() # discard header
        lines = f.read().splitlines()
    rv = OrderedDict()
    for l in lines:
        alm, a, b = l.split(",")
        rv[alm] = (int(a),int(b))
    return rv
    
def compare_alms(ref, old):
    ralm = open_alm(ref)
    oalm = open_alm(old)
    new = OrderedDict()
    diff = []
    for alm in ralm:
        not1, aud1 = ralm[alm]
        try:
            aud2, ign = oalm[alm]
            not2 = not ign
            aud2 = not not aud2
        except KeyError:
            #print("Alarm not found:", alm)
            new[alm] = (not2, aud2)
            continue
        if aud1 != aud2 or aud1 != aud2:
            #print("Diff: Old: aud %s notify %s Ref aud %s notify %s"%(aud2, not2, aud1, not1))
            print("Update: %-35s -> %-6s %s" % (alm, not2, aud2))
            diff.append((alm, not2, aud2))
        new[alm] = (not2, aud2)
    for key in oalm:
        if key not in new:
            #print("Missing: %s"%key)
            pass

In [57]:
import lxml.etree
def calfactors(fp):
    for f in os.listdir(path):
        if f.startswith(fp):
            fp = f
            break
    else:
        raise FileNotFoundError(f)

    if fp.startswith("0003"):
        sz = "3"
    elif fp.startswith("0015"):
        sz = "15"
    elif fp.startswith("0080"):
        sz = "80"
    else:
        raise ValueError(fp)
    refpth = join(dfp%sz, "config", "Cal Factors.cfg")
    oldpth = join(fp, "config", "Cal Factors.cfg")
    
    compare_cals(refpth, oldpth)

class MyXML(hello3.HelloXML):
    def _init(self, root):
        self.data = self.begin_parse(root)

def open_cals(fp):
    with open(fp, 'r') as f:
        txt = f.read()
    return MyXML(txt).data['Cluster']

def compare_cals(ref, old):
    rcal = open_cals(ref)
    ocal = open_cals(old)
    diff = []
    new = OrderedDict()
    for key in rcal:
        rd = rcal[key]
        try:
            od = ocal[key]
        except KeyError:
            #print("Key not found:", key)
            new[key] = rd
            continue
        if type(rd) != type(od):
            raise ValueError("type mismatch for", key)
        if isinstance(rd, str):
            if rd != od:
                print("Update: %-35s -> %s" % (key, od))
        elif isinstance(rd, dict):
            for k1,v1 in rd.items():
                od[k1] != v1
                print("Update: %-35s -> %s" % ("%s %s"%(key, k1), od[k1]))

In [58]:
import lxml.etree
def sysvars(fp):
    for f in os.listdir(path):
        if f.startswith(fp):
            fp = f
            break
    else:
        raise FileNotFoundError(fp)

    if fp.startswith("0003"):
        sz = "3"
    elif fp.startswith("0015"):
        sz = "15"
    elif fp.startswith("0080"):
        sz = "80"
    else:
        raise ValueError(fp)
    refpth = join(dfp%sz, "config", "System Variables.sys")
    oldpth = join(fp, "config", "System Variables.cfg")
    
    compare_sys(refpth, oldpth)
    
def open_sys(fp):
    with open(fp, 'r') as f:
        txt = f.read()
    d = MyXML(txt).data
    c = d['Cluster']
    c.pop("NumElts")
    c.pop("Name")
    return c

def compare_sys(ref, old):
    rsys = open_sys(ref)
    osys = open_sys(old)
    diff = []
    new = OrderedDict()
    for grp in rsys:
        for key in rsys[grp]:
            val = rsys[grp][key]
            try:
                val2 = osys[grp][key]
            except KeyError:
                #print("Key Not Found: %s %s -> %s" %(grp, key, val))
                new[(grp, key)] = val
                continue
            if val2 != val:
                print("Update: %-35s -> %s"%("%s %s"%(grp, key), val2))
            new[(grp,key)] = val2
    for grp in osys:
        for key in osys[grp]:
            if (grp, key) not in new:
                pass
                #print("Missing Key: %s %s -> %s"%(grp, key, osys[grp][key]))

In [73]:
import lxml.etree
def loggersettings(fp):
    for f in os.listdir(path):
        if f.startswith(fp):
            fp = f
            break
    else:
        raise FileNotFoundError(fp)

    if fp.startswith("0003"):
        sz = "3"
    elif fp.startswith("0015"):
        sz = "15"
    elif fp.startswith("0080"):
        sz = "80"
    else:
        raise ValueError(fp)
    refpth = join(dfp%sz, "config", "Logger Settings.log")
    oldpth = join(fp, "config", "Logger Settings.cfg")
    
    compare_log(refpth, oldpth)
    
def open_log(fp):
    with open(fp, 'r') as f:
        f.readline()
        txt = f.read()
    lines = txt.splitlines()
    rv = OrderedDict()
    for line in lines:
        var, thresh, record, grp = line.split("\t")
        rv[var] = (float(thresh), record == "TRUE", grp)
    return rv

def compare_log(ref, old):
    rlog = open_log(ref)
    olog = open_log(old)
    diff = []
    new = OrderedDict()
    for key in rlog:
        t, r, g = rlog[key]
        try:
            t2, r2, g2 = olog[key]
        except KeyError:
            #print("Key not found:", key)
            new[key] = t,r,g
            continue
        if r2 != r or (r2 == True and (t2 != t or g2 != g)):
            print("Update: %-35s -> %s %s (was %s %s)" % (key, t2, r2, t, r))
        new[key] = t2, r2, g2
    for key in olog:
        if key not in new:
            #print("New Key: %s -> %s %s %s"%(key, t2, r2, g2))
            pass

In [74]:
buf = None
import io, builtins, clipboard
def print(*args):
    global buf
    builtins.print(*args, file=buf)
def run(fp):
    global buf
    buf = io.StringIO()
    print("Alarms")
    alms(fp)
    print("\nSys Vars")
    sysvars(fp)
    print("\nCal Factors")
    calfactors(fp)
    print("\nLogger Settings")
    loggersettings(fp)
    clipboard.copy(buf.getvalue())
    builtins.print(buf.getvalue())

In [84]:
run("001516U")

Alarms
Alarms On:

Alarms Off:

Sys Vars
Update: Gas Data PWM On Time (s)            -> 3.0
Update: Gas Data PWM Max Period (s)         -> 60.0
Update: Gas Data Mismatch Thresh (V)        -> 0.2
Update: pH CO2 I Time (min)                 -> 2.0
Update: pH Base I Time (min)                -> 10.0
Update: pH Base P Gain (%/pH)               -> 10.0
Update: pH CO2 Auto Max (%)                 -> 30.0
Update: pH CO2 P Gain (%/pH)                -> -200.0
Update: pH CO2 Manual Max (%)               -> 100.0
Update: pH Deadband                         -> 0.2
Update: DO O2 P Gain (%/DO%)                -> 1.0
Update: DO N2 I Time (min)                  -> 10.0
Update: DO N2 Manual Max (%)                -> 0.0
Update: DO O2 I Time (min)                  -> 10.0
Update: DO N2 P Gain (%/DO%)                -> -5.0
Update: DO N2 Auto Max (%)                  -> 0.0
Update: Process Alarms pH High High         -> 7.6
Update: Process Alarms pH Low Low           -> 6.8
Update: Process Alarms pH Low